## gensim으로 네이버 기사 토픽 모델링 해보기

> 토픽 모델링을 적용하기 위해 텍스트를 처리합니다.

> 토픽 모델링 라이브러리인 gensim을 사용해봅니다.

### 1. 토픽 모델링을 위한 라이브러리 불러오기

In [1]:
#!pip install gensim

In [2]:
# progress bar
from tqdm.notebook import tqdm
# Mecab, Okt 등 형태소 분석기 불러오기
import MeCab
# 특수문자
import string
# 경고 알림 제거를 위한 라이브러리
import warnings
# gensim에서 사용하는 vectorizer 모듈과, LDA model을 불러온다.
from gensim import corpora
from gensim import models


import numpy as np
import re
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings("ignore", category=DeprecationWarning) # 경고 알림이 뜨면 모두 무시합니다.

C:\Users\colin\anaconda3\envs\DH\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


### 2. 텍스트 전처리 함수 만들기

In [3]:
def mecab_nouns(text):
    nouns = []
    
    # 우리가 원하는 TOKEN\tPOS의 형태를 추출하는 정규표현식.
    pattern = re.compile(".*\t[A-Z]+")
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    temp = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]
    
    # 추출한 token중에 POS가 명사 분류에 속하는 토큰만 선택.
    for token in temp:
        if token[1] == "NNG" or token[1] == "NNP" or token[1] == "NNB" or token[1] == "NNBC" or token[1] == "NP" or token[1] == "NR" :
            nouns.append(token[0])
    
    return nouns

def mecab_morphs(text):
    morphs = []
    
    # 우리가 원하는 TOKEN\tPOS의 형태를 추출하는 정규표현식.
    pattern = re.compile(".*\t[A-Z]+")
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    temp = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]
    
    # 추출한 token중에 문자열만 선택.
    for token in temp:
        morphs.append(token[0])
    
    return morphs

# mecab.pos() # MAC
def mecab_pos(text):
    pos = []
    # 우리가 원하는 TOKEN\tPOS의 형태를 추출하는 정규표현식.
    pattern = re.compile(".*\t[A-Z]+")
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    pos = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]    
    
    return pos

In [4]:
def read_documents(input_file_name):
    
    corpus = []
    
    # pk 파일을 읽어서 리스트로 변환하여 돌려줌.
    with open(input_file_name, 'rb') as f:
        temp_corpus = pickle.load(f)
    
    for page in temp_corpus:
        corpus += page
    
    return corpus

def text_cleaning(docs):
    # 한국어를 제외한 글자를 제거하는 함수를 편의를 위해 조금 변형해보자.
    cleaned_docs = []
    
    for doc in docs:
        temp_doc = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", doc)
        cleaned_docs.append(temp_doc)
        
    return cleaned_docs

def define_stopwords(path):
    
    SW = set()
    # 불용어를 추가하는 방법 1.
    # 특수 문자를 추가해보자.
    for i in string.punctuation:
        SW.add(i)
    
    # 불용어를 추가하는 방법 2.
    # stopwords-ko.txt에 직접 추가
    
    with open(path, encoding='UTF8') as f:
        for word in f:
            SW.add(word)

    return SW


def text_tokenizing(corpus, tokenizer):
    # 명사 추출 / 형태소 분석 두 가지를 선택할 수 있게 만들어주는 함수를 만들어보자.
    token_corpus = []
        
    # tqdm을 사용하여 진행 과정을 볼 수 있게 만들어보자.
    if tokenizer == "noun" :
        for n in tqdm(range(len(corpus)), desc="Preprocessing"):
            token_text = mecab_nouns(corpus[n])
            token_text = [word for word in token_text if word not in SW and len(word)>1]
            token_corpus.append(token_text)
        
    elif tokenizer == "morph" :
        for n in tqdm(range(len(corpus)), desc="Preprocessing"):
            token_text = mecab.morphs(corpus[n])
            token_text = [word for word in token_text if word not in SW and len(word)>1]
            token_corpus.append(token_text)
    
    elif tokenizer == "word" :
        for n in tqdm(range(len(corpus)), desc="Preprocessing"):
            token_text = corpus[n].split()
            token_text = [word for word in token_text if word not in SW and len(word)>1]
            token_corpus.append(token_text) 
            
            
    return token_corpus


# 함수를 불러오는 (메인) 코드.
mecab = MeCab.Tagger()
input_file_name = "naver_news_content.pk"
documents = read_documents(input_file_name)
SW = define_stopwords("../Reference/stopwords-ko.txt")
cleaned_text = text_cleaning(documents)
tokenized_text = text_tokenizing(cleaned_text, tokenizer="noun") #tokenizer= "noun" or "morph" or "word"

Preprocessing:   0%|          | 0/9 [00:00<?, ?it/s]

문서 읽기의 과정은 앞서 단어 임베딩의 경우와 다르지 않다. 다음 과정은 문서-단어 행렬을 만드는 과정이다.

In [5]:
# 결과 확인.
print(tokenized_text[0])

['본문', '내용', '플레이어', '플레이어', '오류', '우회', '함수', '추가', '광주', '과학', '기술원', '디지털', '트레이닝', '교육', '참여', '올리브', '네트', '웍스', '광주', '과학', '기술원', '디지털', '트레이닝', '교육', '참여', '인공지능', '분야', '인재', '양성', '지원', '올리브', '네트', '웍스', '국판', '뉴딜', '사업', '일환', '고용', '노동부', '주관', '디지털', '직업훈련', '사업자', '선정', '디지털', '직업훈련', '플랫', '트레이닝', '레딧', '가지', '구성', '올리브', '네트', '웍스', '트레이닝', '교육', '사업', '참여', '디지털', '트레이닝', '문제', '해결', '능력', '향상', '훈련', '방식', '활용', '디지털', '기술', '분야', '핵심', '인력', '양성', '교육', '과정', '기업', '수요', '조사', '실제', '기업', '추진', '비즈니스', '중심', '프로젝트', '설계', '산업', '투입', '가능', '인재', '양성', '성공', '디지털', '트레이닝', '올리브', '네트', '웍스', '포함', '메가', '클라우드', '드림', '에이스', '인공지능', '협회', '컨소시엄', '구성', '이번', '과정', '오프라인', '결합', '중급', '과정', '파이썬', '관련', '기초', '지식', '교육', '선발', '현장', '중심', '교육', '제공', '정규', '교과', '과정', '클라우드', '데이터', '융합', '비즈니스', '문제', '해결', '프로젝트', '과정', '시간', '동안', '참가', '직무', '능력', '향상', '지원', '올리브', '네트', '웍스', '드라마', '나빌레라', '지원', '시청자', '입감', '감동', '선사', '페이스', '서비스', '원료', '영양소', '기반', '식품', '추천

### 3. 토픽 모델링에 사용할 함수들 확인하기

In [6]:
# 문서-단어 행렬 만들기
# 어휘(vocabulary) 학습
dictionary = corpora.Dictionary(tokenized_text)
# 문서-단어 행렬(document-term matrix) 생성
corpus = [dictionary.doc2bow(text) for text in tokenized_text]

In [7]:
# Dictionary 확인
print(dictionary)

Dictionary(448 unique tokens: ['가능', '가지', '감동', '결합', '고민']...)


In [8]:
# corpus 확인
corpus[0][:5]

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2)]

> 인덱스0 단어가 1개, 인덱스1 단어가 1개, 인덱스2 단어가 1개 ....

In [9]:
# TFIDF 문서-단어 행렬 생성
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
corpus_tfidf[0][:5]

[(0, 0.022969325701682166),
 (1, 0.037058199135285304),
 (2, 0.10040960899420624),
 (3, 0.037058199135285304),
 (4, 0.07411639827057061)]

In [10]:
# LDA model 만들기
model = models.ldamodel.LdaModel(corpus, num_topics=3, id2word=dictionary)

In [11]:
# LDA 결과 확인
# model.show_topic(topic_no, num_words)
# 0번째 토픽의 탑 10개를 보고싶다.
model.show_topic(0, 10)

[('디지털', 0.017715793),
 ('교육', 0.013829703),
 ('트레이닝', 0.013799378),
 ('네트', 0.012787113),
 ('웍스', 0.011670766),
 ('올리브', 0.010956337),
 ('사업', 0.0109260725),
 ('과정', 0.010360417),
 ('기업', 0.0095984265),
 ('인공지능', 0.0093853865)]

### 4. 토픽 모델링을 추가하여 코드 완성하기

In [12]:
# 토픽 개수, 키워드 개수를 정해주는 변수를 추가.
NUM_TOPICS = 3
NUM_TOPIC_WORDS = 30

def build_doc_term_mat(documents):
    # 문서-단어 행렬 만들어주는 함수.
    print("Building document-term matrix.")
    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]
    
    return corpus, dictionary


def print_topic_words(model):
    # 토픽 모델링 결과를 출력해 주는 함수.
    print("\nPrinting topic words.\n")    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))     
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
        print()

# document-term matrix를 만들고,
corpus, dictionary = build_doc_term_mat(tokenized_text)
# LDA를 실행.
model = models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha='auto', eta='auto') # passes : 일반적으로 1000번
# 결과를 출력.
print_topic_words(model)

Building document-term matrix.

Printing topic words.

Topic ID: 0
	교육	0.018670478835701942
	개발	0.014757138676941395
	기술	0.012646947987377644
	디지털	0.011375480331480503
	서비스	0.010086036287248135
	기업	0.009648780338466167
	공부	0.00934840738773346
	올리브	0.00879703275859356
	네트	0.008765244856476784
	프로젝트	0.008648534305393696
	대위	0.008598621003329754
	내용	0.008408005349338055
	사업	0.008321921341121197
	웍스	0.007896640338003635
	제공	0.007507308851927519
	과정	0.00732471002265811
	의료	0.007148123811930418
	육군	0.006399653386324644
	트레이닝	0.00629810756072402
	양성	0.006266407668590546
	인공지능	0.006214702967554331
	지원	0.00617619464173913
	추천	0.00596906803548336
	관련	0.005790397524833679
	본문	0.005780673120170832
	뉴스	0.005628587212413549
	분야	0.005408165976405144
	시간	0.005276098381727934
	사단	0.005188449751585722
	과학	0.005180715583264828

Topic ID: 1
	교육	0.023130137473344803
	디지털	0.022333431988954544
	올리브	0.020252512767910957
	웍스	0.019105875864624977
	과정	0.017526334151625633
	네트	0.01750141754746437
	트레이닝	0.01711092

### 5. pyLDAvis를 통한 토픽 모델링 결과 시각화하기

In [13]:
#!pip install pyldavis

In [14]:
# pyLDAvis 불러오기
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# pyLDAvis를 jupyter notebook에서 실행할 수 있게 활성화.
pyLDAvis.enable_notebook()
# pyLDAvis 실행.
gensimvis.prepare(model, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.031930 -0.000593       1        1  69.687509
0     -0.017022 -0.013140       2        1  17.168478
2     -0.014908  0.013733       3        1  13.144013, topic_info=     Term       Freq      Total Category  logprob  loglift
11     교육  43.000000  43.000000  Default  30.0000  30.0000
45     사업  20.000000  20.000000  Default  29.0000  29.0000
19     기업  19.000000  19.000000  Default  28.0000  28.0000
127    개발  12.000000  12.000000  Default  27.0000  27.0000
17     기술  19.000000  19.000000  Default  26.0000  26.0000
..    ...        ...        ...      ...      ...      ...
6      과정   1.771623  28.013534   Topic3  -4.9735  -0.7316
17     기술   1.523516  19.388866   Topic3  -5.1243  -0.5145
60     시간   1.408843  12.952099   Topic3  -5.2026  -0.1893
102    추가   1.271467   7.820607   Topic3  -5.3052   0.2126
106  클라우드   1.292357  13.749946   Topic3  -5.2889  -0.3354

[215 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
0         1  0.740967    가능
0         2  0.092621    가능
0         3  0.092621    가능
300       1  0.598400   강상규
300       3  0.598400   강상규
...     ...       ...   ...
122       1  0.876631    협회
122       2  0.219158    협회
298       1  0.873884  홈페이지
261       1  0.531918    환자
261       2  0.265959    환자

[336 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3])